In [1]:
# 注意：900条训练集 这是一个处理数据集，生成最大内接矩形的脚本
import os
import cv2
import numpy as np
import csv


# 获取数据集路径
image_dir = '../data/ISIC/' # 图像目录
print(image_dir)

# 设置各子目录路径
train_image_dir = os.path.join(image_dir, "ISBI2016_ISIC_Part1_Training_Data")
test_image_dir = os.path.join(image_dir, "ISBI2016_ISIC_Part1_Test_Data")
train_mask_dir = os.path.join(image_dir, "ISBI2016_ISIC_Part1_Training_GroundTruth")
test_mask_dir = os.path.join(image_dir, "ISBI2016_ISIC_Part1_Test_GroundTruth")

output_file = os.path.join(image_dir, "processed_annotations.csv")

# 获取训练集和测试集文件
train_image_files = sorted([f for f in os.listdir(train_image_dir) if f.endswith(".jpg")])
test_image_files = sorted([f for f in os.listdir(test_image_dir) if f.endswith(".jpg")])
train_mask_files = sorted([f for f in os.listdir(train_mask_dir) if f.endswith(".png")])
test_mask_files = sorted([f for f in os.listdir(test_mask_dir) if f.endswith(".png")])
print(f"训练集图像数量: {len(train_image_files)}",train_image_files[0])
print(f"测试集图像数量: {len(test_image_files)}",test_image_files[0])

# 合并所有文件
all_image_files = train_image_files + test_image_files
all_mask_files = train_mask_files + test_mask_files

# 检查文件数量和匹配性
assert len(all_image_files) == len(all_mask_files), \
    f"图像和掩码数量不匹配！图像数: {len(all_image_files)}, 掩码数: {len(all_mask_files)}"

../data/ISIC/
训练集图像数量: 900 ISIC_0000000.jpg
测试集图像数量: 379 ISIC_0000003.jpg


In [3]:

# 找最大内接矩形（直方图法）
def get_max_inner_rectangle(mask, x_offset=0, y_offset=0):
    mask_bin = (mask > 0).astype(np.uint8)
    row, col = mask_bin.shape
    height = [0] * (col + 2)
    max_area = 0
    result_rect = None
    
    for i in range(row):
        stack = []
        for j in range(col + 2):
            if 1 <= j <= col:
                height[j] = height[j] + 1 if mask_bin[i][j - 1] == 1 else 0
            while stack and height[stack[-1]] > height[j]:
                cur = stack.pop()
                area = (j - stack[-1] - 1) * height[cur]
                if area > max_area:
                    max_area = area
                    left = stack[-1] + 1
                    right = j - 1
                    top = i - height[cur] + 1
                    bottom = i
                    result_rect = [left + x_offset, top + y_offset, 
                                  right + x_offset, bottom + y_offset]
            stack.append(j)
    return result_rect


In [4]:
# 输出文件路径
output_file_train = "train_boxes_adaptive.csv"
output_file_test = "test_boxes_adaptive.csv"

# 写入两个文件的表头
for file in [output_file_train, output_file_test]:
    with open(file, "w", newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow([
            "image_file", "mask_file", 
            "max_boxes_x1", "max_boxes_y1", "max_boxes_x2", "max_boxes_y2",  # 最小外接矩形
            "min_boxes_x1", "min_boxes_y1", "min_boxes_x2", "min_boxes_y2"   # 最大内接矩形
        ])

processed_train, processed_test = 0, 0

# 遍历图像列表
for image_file in all_image_files:
    if image_file in train_image_files:
        image_path = os.path.join(train_image_dir, image_file)
        mask_path = os.path.join(train_mask_dir, image_file.replace(".jpg", "_Segmentation.png"))
        output_file = output_file_train
    else:
        image_path = os.path.join(test_image_dir, image_file)
        mask_path = os.path.join(test_mask_dir, image_file.replace(".jpg", "_Segmentation.png"))
        output_file = output_file_test

    print(mask_path)
    
    # 读取图像和掩码
    image = cv2.imread(image_path)
    if image is None:
        print(f"警告: 无法读取图像 {image_path}")
        continue
        
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    if mask is None:
        print(f"警告: 无法读取掩码 {mask_path}")
        continue

    # 最小外接矩形
    x, y, w, h = cv2.boundingRect(mask)
    min_rect = [x, y, x + w, y + h]

    # 最大内接矩形
    max_inner_rect = get_max_inner_rectangle(mask)
    
    if max_inner_rect is None:
        print(f"警告: 未找到图像 {image_file} 的最大内接矩形，使用最小外接矩形替代")
        max_inner_rect = min_rect.copy()

    # 追加写入对应CSV
    with open(output_file, "a", newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow([
            image_file, 
            os.path.basename(mask_path),
            *min_rect,
            *max_inner_rect
        ])

    # 统计数量
    if output_file == output_file_train:
        processed_train += 1
    else:
        processed_test += 1
    print(f"处理完成: {image_file}")

# 完成提示
print(f"\n✅ 所有处理完成！共处理训练集 {processed_train} 张，测试集 {processed_test} 张图像")
print(f"训练集标注保存至：{output_file_train}")
print(f"测试集标注保存至：{output_file_test}")


../data/ISIC/ISBI2016_ISIC_Part1_Training_GroundTruth/ISIC_0000000_Segmentation.png


处理完成: ISIC_0000000.jpg
../data/ISIC/ISBI2016_ISIC_Part1_Training_GroundTruth/ISIC_0000001_Segmentation.png
处理完成: ISIC_0000001.jpg
../data/ISIC/ISBI2016_ISIC_Part1_Training_GroundTruth/ISIC_0000002_Segmentation.png
处理完成: ISIC_0000002.jpg
../data/ISIC/ISBI2016_ISIC_Part1_Training_GroundTruth/ISIC_0000004_Segmentation.png
处理完成: ISIC_0000004.jpg
../data/ISIC/ISBI2016_ISIC_Part1_Training_GroundTruth/ISIC_0000006_Segmentation.png
处理完成: ISIC_0000006.jpg
../data/ISIC/ISBI2016_ISIC_Part1_Training_GroundTruth/ISIC_0000007_Segmentation.png
处理完成: ISIC_0000007.jpg
../data/ISIC/ISBI2016_ISIC_Part1_Training_GroundTruth/ISIC_0000008_Segmentation.png
处理完成: ISIC_0000008.jpg
../data/ISIC/ISBI2016_ISIC_Part1_Training_GroundTruth/ISIC_0000009_Segmentation.png


KeyboardInterrupt: 